# PaddleCamp_第二期

## 第二周 - 第一次作业参考答案


该作业共 **1** 部分：

----
* 代码填空题 **参考答案**：

In [1]:
#因为本次实验除了代码还有许多知识点讲解。
#知识点是以markdown形式展现的，含有许多图片。
#运行完这条命令，并刷新一下本页面，本实验中的图片就可以展现出来了。
#这条命令只需要运行一遍就可以了。
!DATA_PATH=data/data7141/ && NEW_NAME=$(find -name *[0-9].ipynb) && NEW_NAME=${NEW_NAME%.*} && NEW_NAME=${NEW_NAME#./} && unzip -o ${DATA_PATH}cat_logist.zip  && cp -rf cat_logist/. .

In [2]:
import sys
import numpy as np

import lr_utils
import matplotlib.pyplot as plt

import paddle
import paddle.fluid as fluid

from paddle.utils.plot import Ploter
%matplotlib inline


# 调用load_dataset()函数，读取数据集
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = lr_utils.load_dataset()

# 图片示例
# 可观察到索引为“23”的图片应为“non-cat”
index = 23
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

In [3]:
### START CODE HERE ### (≈ 3 lines of code)
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

### END CODE HERE ###

print ("训练样本数: m_train = " + str(m_train))
print ("测试样本数: m_test = " + str(m_test))
print ("图片高度/宽度: num_px = " + str(num_px))
print ("图片大小: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [4]:
# 定义维度
DATA_DIM = num_px * num_px * 3

# 转换数据形状

### START CODE HERE ### (≈ 2 lines of code)
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1)
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1)
### END CODE HERE ###

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [5]:
### START CODE HERE ### (≈ 2 lines of code)
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

### END CODE HERE ###

train_set = np.hstack((train_set_x, train_set_y.T))
test_set = np.hstack((test_set_x, test_set_y.T))

# 读取训练数据或测试数据
def read_data(data_set):
    """
        一个reader
        Args:
            data_set -- 要获取的数据集
        Return:
            reader -- 用于获取训练数据集及其标签的生成器generator
    """
    def reader():
        """
        一个reader
        Args:
        Return:
            data[:-1], data[-1] -- 使用yield返回生成器(generator)，
            data[:-1]表示前n-1个元素组成的list，也就是训练数据，data[-1]表示最后一个元素，也就是对应的标签
        """
        for data in data_set:
            ### START CODE HERE ### (≈ 1 lines of code)
            yield data[:-1], data[-1]
            ### END CODE HERE ###
    return reader

test_array = [[1,1,1,1,0],
                [2,2,2,2,1],
                [3,3,3,3,0]]

print("test_array for read_data:")
for value in read_data(test_array)():
    print(value)
    
BATCH_SIZE=200

# 设置训练reader
train_reader = paddle.batch(
    paddle.reader.shuffle(
        read_data(train_set), buf_size=500),
    batch_size=BATCH_SIZE)

#设置测试 reader
test_reader = paddle.batch(
    paddle.reader.shuffle(
        read_data(test_set), buf_size=500),
    batch_size=BATCH_SIZE)
    
    

In [6]:
BATCH_SIZE=200

# 设置训练reader
train_reader = paddle.batch(
    paddle.reader.shuffle(
        read_data(train_set), buf_size=500),
    batch_size=BATCH_SIZE)

#设置测试 reader
test_reader = paddle.batch(
    paddle.reader.shuffle(
        read_data(test_set), buf_size=500),
    batch_size=BATCH_SIZE)
    
    
### START CODE HERE ### (≈ 3 lines of code) 
x = fluid.layers.data(name='x', shape=[DATA_DIM], dtype='float32')
y_predict = fluid.layers.fc(input=x, size=2, act='softmax') 
y = fluid.layers.data(name='y', shape=[1], dtype='int64') 
### END CODE HERE ###   

cost = fluid.layers.cross_entropy(input=y_predict, label=y)
avg_loss = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=y_predict, label=y)

startup_program = fluid.default_startup_program()
main_program = fluid.default_main_program()

#克隆main_program得到test_program
#有些operator在训练和测试之间的操作是不同的，例如batch_norm，使用参数for_test来区分该程序是用来训练还是用来测试
#该api不会删除任何操作符,请在backward和optimization之前使用
test_program = fluid.default_main_program().clone(for_test=True)

optimizer = fluid.optimizer.Adam(learning_rate=0.001)
optimizer.minimize(avg_loss)
print("optimizer is ready")

# 使用CPU训练
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

exe = fluid.Executor(place)

save_dirname="recognize_cat_inference.model"

train_prompt = "Train cost"
cost_ploter = Ploter(train_prompt)

In [7]:
# 将训练过程绘图表示
def event_handler_plot(ploter_title, step, cost):
    cost_ploter.append(ploter_title, step, cost)
    cost_ploter.plot()
    
    
def train_test(train_test_program, train_test_feed, train_test_reader):
    # 将分类准确率存储在acc_set中
    acc_set = []
    # 将平均损失存储在avg_loss_set中
    avg_loss_set = []
    # 将测试 reader yield 出的每一个数据传入网络中进行训练
    for test_data in train_test_reader():
        acc_np, avg_loss_np = exe.run(
            program=train_test_program,
            feed=train_test_feed.feed(test_data),
            fetch_list=[acc, avg_loss])
        acc_set.append(float(acc_np))
        avg_loss_set.append(float(avg_loss_np))
    
    # get test acc and loss
     # 获得测试数据上的准确率和损失值
    acc_val_mean = np.array(acc_set).mean()
    avg_loss_val_mean = np.array(avg_loss_set).mean()
    # 返回平均损失值，平均准确率
    return avg_loss_val_mean, acc_val_mean

In [8]:
feeder = fluid.DataFeeder(place=place, feed_list=[x, y])
exe.run(startup_program)

PASS_NUM = 150
epochs = [epoch_id for epoch_id in range(PASS_NUM)]


lists = []

step = 0
for epoch_id in epochs:
    for step_id, data in enumerate(train_reader()):
        metrics = exe.run(
            main_program,
            feed=feeder.feed(data),
            fetch_list=[avg_loss,acc])
        #我们可以把训练结果打印输出，也可以用画图展示出来
        if step % 10 == 0: #每训练10次，绘制一次曲线
            event_handler_plot(train_prompt, step, metrics[0])
        step += 1

    # 测试每个epoch的分类效果
    avg_loss_val, acc_val = train_test(
        train_test_program=test_program,
        train_test_reader=test_reader,
        train_test_feed=feeder)

    print("Test with Epoch %d, avg_cost: %s, acc: %s" %
          (epoch_id, avg_loss_val, acc_val))
    lists.append((epoch_id, avg_loss_val, acc_val))
    
    # 保存训练好的模型参数用于预测
    if save_dirname is not None:
        fluid.io.save_inference_model(save_dirname, ["x"], [y_predict], exe)

# 选择效果最好的pass
best = sorted(lists, key=lambda list: float(list[1]))[0]
print('Best pass is %s, testing Avgcost is %s' % (best[0], best[1]))
print('The classification accuracy is %.2f%%' % (float(best[2]) * 100))


In [9]:
#指定预测的作用域
inference_scope = fluid.core.Scope()
with fluid.scope_guard(inference_scope):

    # 使用 fluid.io.load_inference_model 获取 inference program,
    # feed_target_names 用于指定需要传入网络的变量名
    # fetch_targets 指定希望从网络中fetch出的变量名
    [inference_program, feed_target_names,
     fetch_targets] = fluid.io.load_inference_model(
         save_dirname, exe)

    # 将feed构建成字典 {feed_target_name: feed_target_data}
    # 结果将包含一个与fetch_targets对应的数据列表
    # 我们可以实现批量预测，通过一个循环，每次预测一个mini_batch
    for mini_batch in test_reader():
        test_x = np.array([data[0] for data in mini_batch]).astype("float32")
        test_y = np.array([data[1] for data in mini_batch]).astype("int64")
        # 真实进行预测
        mini_batch_result = exe.run(
            inference_program,
            feed={feed_target_names[0]: test_x},
            fetch_list=fetch_targets)
        
        # 打印预测结果
        mini_batch_result = np.argsort(mini_batch_result) #找出可能性最大的列标，升序排列
        mini_batch_result = mini_batch_result[0][:, -1]  #把这些列标拿出来
        print('预测结果：%s'%mini_batch_result)

        # 打印真实结果  
        label = test_y # 转化为 label
        print('真实结果：%s'%label)
        
    #计数
    label_len = len(label)
    right_counter = 0
    for i in range(label_len):
        if mini_batch_result[i] == label[i]:
            right_counter += 1
    
    ratio = (right_counter/label_len)
    print("准确率为：%.2f%%"%(ratio*100))